In [37]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [38]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [39]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [40]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [41]:
# Define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        # Flatten the input image from 28x28 to 784
        self.flatten = nn.Flatten()

        # 784 input features, 64 output features
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=10, bias=True)
    (7): ReLU()
  )
)


In [42]:
for param in model.parameters():
    print(param.shape)

torch.Size([512, 784])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([10, 512])
torch.Size([10])


In [43]:
# Optimizing the Model Parameters

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [44]:
def train(dataloader, model, loss_fn, optimizer):
    
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):

        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [45]:
def test(data_loader, model, loss_fn):

    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in data_loader:

            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [48]:
# The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.211623  [   64/60000]
loss: 2.219437  [ 6464/60000]
loss: 2.178976  [12864/60000]
loss: 2.127076  [19264/60000]
loss: 2.157579  [25664/60000]
loss: 2.205688  [32064/60000]
loss: 2.083064  [38464/60000]
loss: 2.137163  [44864/60000]
loss: 2.141768  [51264/60000]
loss: 2.053479  [57664/60000]
Test Error: 
 Accuracy: 37.8%, Avg loss: 2.087874 

Epoch 2
-------------------------------
loss: 2.164294  [   64/60000]
loss: 2.171742  [ 6464/60000]
loss: 2.112392  [12864/60000]
loss: 2.019309  [19264/60000]
loss: 2.078426  [25664/60000]
loss: 2.161372  [32064/60000]
loss: 1.967560  [38464/60000]
loss: 2.057731  [44864/60000]
loss: 2.065075  [51264/60000]
loss: 1.934729  [57664/60000]
Test Error: 
 Accuracy: 38.0%, Avg loss: 1.992295 

Epoch 3
-------------------------------
loss: 2.109200  [   64/60000]
loss: 2.114212  [ 6464/60000]
loss: 2.040047  [12864/60000]
loss: 1.900954  [19264/60000]
loss: 1.993608  [25664/60000]
loss: 2.115875  [32064/600

In [49]:
# saving and loading model

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [50]:
# Load model

model_load = NeuralNetwork().to(device)
model_load.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [56]:
import numpy as np

In [75]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
i = np.random.randint(0, len(test_data))
x, y = test_data[i][0], test_data[i][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Coat"
